# 정수 인코딩 ( Integer Encoding)
- 컴퓨터는 텍스트보다는 숫자를 더 잘 처리할 수 있다. 따라서 텍스트를 숫자로 바꿔주는 여러 기법들이 있다.
- 첫 단계로 각 단어를 고유한 정수에 매핑(mapping)시켜 주어야 한다.
    - 단어가 5,000개 있다면 1~5000번까지 단어와 매핑시켜준다.
    - 인덱스를 부여할 때, 단어 등장 빈도수를 기준으로 정렬하여 부여한다 (랜덤으로 하기도 함)
    

단어에 정수를 부여하는 방법
1. 빈도수 순으로 정렬한 단어 집합을 만들고
2. 빈도수가 높은 순서대로 차례대로 낮은 숫자부터 정수를 부여


### 1. Dictionary 사용하기

In [2]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

raw_text = "A barber is a person. a barber is good person. a barber is huge person.\
    he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. \
        a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy.\
            the barber went up a huge mountain."

In [3]:
# 기존의 텍스트 데이터가 문장 단위로 토큰화 된 것을 확인할 수 있다.
# 이제 정제 작업 + 정규화 작업을 병행하여, 단어 토큰화 작업을 수행해준다
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [4]:
# 여기서는 단어 소문자화하여 단어의 개수를 통일,
# 불용어와 단어 길이가 2 이하인 경우 단어를 제외

### 주의!
### 텍스트를 수치화하는 단계라는 것은, 자연어 처리 작업에 들어가므로, 최대한의 전처리를 끝내야 한다!!!!!

vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
    # 단어 토큰화
    tokenized_sentence = word_tokenize(sentence)
    result = []
    
    for word in tokenized_sentence:
        word = word.lower() # 모든 문자 소문자화하여 단어 개수 줄이기
        if word not in stop_words: # 단어 토큰화 된 결과에 대해 불용어 처리하기
            if len(word)>2 : #단어 길이가 2 이하인 경우, 추가로 단어 제거
                result.append(word)
                if word not in vocab:
                    vocab[word]=0
                vocab[word]+=1
    preprocessed_sentences.append(result)
print(preprocessed_sentences)
                                                         

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [5]:
# 파이썬의 딕셔너리 구조로 단어를 키(key)로, 단어에 대한 빈도수가(value) 로 저장되어 있다.
print("단어 집합 :", vocab)

# 특정 단어에 대한 빈도수를 조회하고 싶을 때,
print("특정 단어 'barber' 빈도수 :", vocab['barber'])

단어 집합 : {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}
특정 단어 'barber' 빈도수 : 8


In [6]:
# 빈도수 기준으로 정렬하기
sorted_vocab = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(sorted_vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


정수 부여해주기

In [7]:
# 높은 빈도수일수록 낮은 인덱스 부여
word_to_index={}

i=0

for (word, frequency) in sorted_vocab:
    if frequency>1: # 빈도수가 작은 단어는 제외
        i=i+1
        word_to_index[word]=i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


자연어 처리를 하다보면, 빈도수가 가장 높은 n개의 단어들만 사용하고 싶은 경우가 많다


In [8]:
# n 개의 단어만 조회하고 싶을 때
vocab_size = 5

# 인덱스가 5 초과인 단어는 제거
words_frequency = [ word for word, index in word_to_index.items() if index>=vocab_size+1]

# 인덱스가 5 초과인 애들만 추출
print(words_frequency)

# 해당 단어에 대한 인덱스 정보를 삭제
for w in words_frequency:
    del word_to_index[w]
print(word_to_index)

['word', 'keeping']
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


word_to_index 에 이제 상위 5개의 단어만 저장되었다.

- 이제 단어 토큰화가 된 상태로 저장된 sentences에 이쓴ㄴ 각 단어를 정수로 바꾸는 작업이 필요
    - e.x) sentences에서 첫 문장은 ['barber', 'person'] 이었는데, 이 문장은 [1,5] 로 인코딩 된다.
    - 두 번째 문장은 ['barber', 'good', 'person'] 인데 word_to_index에 존재 하지 않는다
        - good 을 보고 Out-of-Vocabulary 문제라고 함 (OOV 문제)
        - 따라서 word_to_index 에 OOV 라는 단어를 새로 추가하여 인코딩해준다

In [9]:
# word_to_index 에 OOV추가
word_to_index['OOV']=len(word_to_index)+1
print(word_to_index)    

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [10]:
encoded_sentences = []

for sentence in preprocessed_sentences:
    encoded_sentence=[]
    for word in sentence:
        try:
            # 단어 집합에 존재하면 해당 단어 인덱스 리턴
            encoded_sentence.append(word_to_index[word])
        except KeyError:
            # 단어 집합에 존재하지 않으면, OOV 인덱스를 리턴
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence)
print(encoded_sentences)
    

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


### 2. Counter 사용하기

In [11]:
from collections import Counter

print(preprocessed_sentences)


[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


단어 집합(vocabulary)를 만들기 위해 sentences 에서 문장 경계인 [, ] 를 제거

In [12]:
# words = np.hstack(preprocessed_sentences) 으로도 가능

all_words_list = sum(preprocessed_sentences, [])
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [13]:
# 파이썬의 counter 모듈로 단어 빈도수 세기

vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [14]:
# 빈도수 높은 순으로 n 개를 보고 싶을 때
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [15]:
# 높은 빈도수를 가진 단어일수록 낮은 인덱스 부여하기
word_to_index = {}

i=1
for word, freq in vocab:
    word_to_index[word]=i
    i+=1
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}

### NLTK의 FreqDist 사용하기
- Counter() 와 같은 방법으로 사용 가능하다

In [20]:
from nltk import FreqDist
import numpy as np

print(preprocessed_sentences, end='\n')
vocab = FreqDist(np.hstack(preprocessed_sentences))
print("문장 구분 제거")
print(vocab)
print(vocab['barber'])


[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
문장 구분 제거
<FreqDist with 13 samples and 36 outcomes>
8


In [21]:
# counter와 동일하게 쓸 수 있다
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [23]:
word_to_index = { word[0] : index+1 for index, word in enumerate(vocab)}
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}

## 케라스(Keras)의 텍스트 전처리


In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [25]:
preprocessed_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [37]:
tokenizer = Tokenizer()

# fit_on_text() 안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성해준다
tokenizer.fit_on_texts(preprocessed_sentences)

print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


fit_on_texts 를 통해 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여해준다
- 아주 편리함

In [38]:
# 각 단어 카운트 했을 때 개수
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [40]:
# 입력된 코퍼스를 각 인덱스에 배정
print(tokenizer.texts_to_sequences(preprocessed_sentences))


[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [43]:
# 상위 5개만 가져오기
vocab_size = 5 
tokenizer = Tokenizer(num_words=vocab_size+1)
tokenizer.fit_on_texts(preprocessed_sentences)

print(tokenizer.word_index)
print()
print(tokenizer.word_counts)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [44]:
# 실제 적용은 texts_to_sequences 을 할때, 적용이 된다.
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [46]:
# OOV 미포함 전체 과정
# 케라스는 기본적으로 단어 집합에 없는 단어인 OOV에 대하여 단어를 정수로 바꾸는 과정에서 단어를 제거하게 된다.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

vocab_size = 5
words_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size+1]

for word in words_frequency:
    del tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보 삭제
    del tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제
    
    
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))


{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [48]:
# OOV 포함 전체 과정
# oov_token 매개변수 사용
vocab_size = 5
tokenizer = Tokenizer(num_words= vocab_size+2, oov_token='OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

# OOV의 인덱스의 기본은 1 이다
print("단어 OOV의 인덱스 : {}".format(tokenizer.word_index['OOV']))

# 코퍼스에 대해서 정수 인코딩
res = tokenizer.texts_to_sequences(preprocessed_sentences)
print(res)

# 빈도수 상위 5개는 2~6 으로 인코딩 // OOV 는 1으로 인코딩!!!

단어 OOV의 인덱스 : 1
[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
